In [2]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import pickle

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [30]:
# train=pd.read_pickle("VDISC_train1.pickle")
# # validate=pd.read_pickle("VDISC_validate.pickle")
# test=pd.read_pickle("VDISC_test1.pickle")

# for dataset in [train,test]:
#     for col in range(1,6):
#         dataset.iloc[:,col] = dataset.iloc[:,col].map({False: 0, True: 1})

# # Create source code sdata for tokenization
# x_all = train['functionSource']
# x_all = [x.decode('utf-8') for x in x_all]
# #x_all = x_all.append(validate['functionSource'])
# #x_all = x_all.append(test['functionSource'])

# # Tokenizer with word-level
# tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=False)
# tokenizer.fit_on_texts(list(x_all))
# del(x_all)

In [4]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer= pickle.load(handle)

In [5]:
tokenizer

In [28]:
#source_code = train['functionSource'][1]
source_code = b'readinterfaces(int fd, section* CSec, section *)\n{\n\tchar name[10];\n\tFILE *iflst;\n\tchar s[BUFSIZ];\n\tchar *p;\n\n\tif ((iflst = fopen(FILE_PROC, ""r"")) == NULL)\n\t{\n\t\tperror(FILE_PROC);\n\t\treturn -1;\n\t}\n\n\twhile (!feof(iflst))\n\t{\n\t\tfgets(s, sizeof(s), iflst);\n\n\t\tif ((p = strchr(s, \':\')))\n\t\t{\n\t\t\t*p = 0;\n\t\t\tsscanf(s, ""%s"", name);\n\t\t\tp = readoptions(fd, name, 1, CSec, PSec);\n\n\t\t\twhile (p != NULL && *p != \'\\0\')\n\t\t\t{\n\t\t\t\tstrcpy(name, p);\n\t\t\t\tp = readoptions(fd, name, 0, CSec, PSec);\n\t\t\t}\n\t\t}\n\t}\n\n\tfclose(iflst);\n\n\treturn 0;\n}'
# print(source_code)
x = source_code.decode('utf-8')
list_tokenized_text = tokenizer.texts_to_sequences(x)
print(x)

readinterfaces(int fd, section* CSec, section *)
{
	char name[10];
	FILE *iflst;
	char s[BUFSIZ];
	char *p;

	if ((iflst = fopen(FILE_PROC, ""r"")) == NULL)
	{
		perror(FILE_PROC);
		return -1;
	}

	while (!feof(iflst))
	{
		fgets(s, sizeof(s), iflst);

		if ((p = strchr(s, ':')))
		{
			*p = 0;
			sscanf(s, ""%s"", name);
			p = readoptions(fd, name, 1, CSec, PSec);

			while (p != NULL && *p != '\0')
			{
				strcpy(name, p);
				p = readoptions(fd, name, 0, CSec, PSec);
			}
		}
	}

	fclose(iflst);

	return 0;
}


In [29]:
data = tf.keras.preprocessing.sequence.pad_sequences(list_tokenized_text,
                                                                 maxlen=500,
                                                                 padding='post',
                                                     dtype=object)
data = data.astype(np.int64)

In [9]:
model = tf.keras.models.load_model("vuldet.h5")


In [41]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 13)      130000      ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 500, 512)     60416       ['embedding[0][0]']              
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 100, 512)     0           ['conv1d[0][0]']                 
                                                                                              

In [30]:
predicted = model.predict(data)

17/17 [==============================] - 0s 11ms/step


In [31]:
# False - 0, True - 1
pred_test = [[],[],[],[],[]]

for col in range(0,len(predicted)):
    for row in predicted[col]:
        if row[0] >= row[1]:
            pred_test[col].append(0)
        else:
            pred_test[col].append(1)
            

for col in range(0,len(pred_test)):
    print(pd.value_counts(pred_test[col]))

0    520
dtype: int64
0    520
dtype: int64
0    520
dtype: int64
0    520
dtype: int64
0    520
dtype: int64


In [24]:
 print(pred_test)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,